#### NX-414: Brain-like computation and intelligence
##### TA: Alessandro Marin Vargas

# Best model usage - Mini projects (Predicting neural activity)

## Import the relevant packages and functions

In [ ]:
from utils import load_it_data
import torch
from torch.models import ResNet
from test import extract_activations
import joblib

## Load the data

In [ ]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, stimulus_val, stimulus_test, objects_train, objects_val, objects_test, spikes_train, spikes_val = load_it_data(path_to_data)

## Load the models
Load the models that have been saved in joblib format (and add the function that extract the activations to the ResNet model).

In [ ]:
resnet = joblib.load('resnet50_finetuned.joblib')
pca = joblib.load('pca_layer3.joblib')
ridge = joblib.load('ridge_layer3.joblib')
ResNet.extract_activations = extract_activations

# Extract the activations from the finetuned ResNet model
Define the function that extract the activations from the layer 3 and add it to the ResNet model.

In [ ]:
def extract_activations(self, stimuli) :
    """extract the activations of the model for the given stimuli

    Args:
        model (model): model we want to extract the activations from
        stimuli (ndarray): input data of the processed image's pixels

    Returns:
        list of ndarray: list of activations for each stimulus
    """    
    activations = []
    for x in stimuli : 
        x = self.conv1(x.unsqueeze(0))
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)        
        x = self.layer3(x)
        activations.append(torch.flatten(x.squeeze(0)).detach().cpu().numpy())
    return activations

activations_val = resnet.extract_activations(torch.tensor(stimulus_val).cuda(), 'layer3')

# Compute the 1000 PCs from the activations

In [ ]:
activations_val = pca.transform(activations_val)

# Make predictions

In [ ]:
predictions = ridge.predict(activations_val)